


# `파일 2`

1. 저장된 yolo파라미터를 적용한 yolo모델을 만든다.

2. 이미지이름에 라벨링 정보가 포함된 이미지 폴더를 제공한다. ex) 아우터_무스탕_image1.jpg

3. 학습 파라미터가 적용된 yolo에 이미지 폴더 이미지를 넣는다.

4. 이미지를 투입시켜 yolo에서 탐지된 바운딩박스와 이미지이름에 포함된 라벨링 정보를 모은다.

5. 이때 바운딩 박스는 effcientnet 인풋에 맞는 사이즈(224x224)로 조정한다.

6. 이때 라벨은 총 40개의 클래스를 가진다.

7. 모아진 바운딩 박스와 라벨 정보를 이용해서 데이터셋을 구축하고 efficientent을 학습시킨다.

8. 원본 이미지는 학습 결과로 원본이미지와 분류된 클래스를 보여줄 것이기 때문에 dataset에 일단 포함한다.

9. efficientnet 학습 파라미터를 저장한다.




# 구글 드라이브 연동 및 import

- import 임시용 : 없어도 되면 삭제할 것

In [1]:
# 구글 드라이브 연동하기
from google.colab import drive
drive.mount('/content/drive')

# 경로 및 파일 관리
import os
import shutil


# 딥러닝 라이브러리
import torch

# 특정 확장자 파일 불러오기 -> glob.glob() : 특정 패턴과 일치하는 모든 경로명을 리스트로 반환
import glob

# 학습한 베스트 모델 다운로드하기
from google.colab import files


# 딥러닝 사용
import torch


# 이미지 관련
from PIL import Image as Img
from PIL import ImageFile
from IPython.display import Image, display # clear_output
import cv2
import numpy as np
import glob


# 파일 관리
import os

# efficientNet_v2_s 모델을 가지고오기 위함
import torchvision.models as get_model
import torchvision

# input 데이터를 정규화하기 위한 transform 규칙을 적용하기 위함
import torchvision.transforms as transforms

# optimizer를 가지고 오기 위함.
import torch.optim as optim

# pytorch에서 지원하는 다양한 계층 및 손실함수 계산을 사용하기 위함.
import torch.nn as nn

from torchvision.datasets import ImageFolder
from torchvision import transforms # ToTensor, Resize, Compose
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader

# 스케쥴러
from torch.optim.lr_scheduler import OneCycleLR

from tqdm import tqdm

Mounted at /content/drive


### 저장된 학습 yolo 모델 불러오기

In [2]:
# 학습된 yolo pt 경로 설정하기
yolov5_best_pt_save_path = '/content/drive/MyDrive/yolo_efficientnet/yolo_best.pt/final_fashion_yolo_best2.pt'

# 학습 파라미터를 적용한 모델 불러오기final_fashion_yolo_best2.pt
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path = yolov5_best_pt_save_path )

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 140.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 253.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 152.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 7.1s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-7-3 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


### yolo input으로 넣기 위한 custom 이미지 폴더 경로 지정하기
- 해당 이미지 폴더는 다음과 같이 사이즈가 정규화 되어져 있어야 한다.
    - 1. (yolo) 416 (비율 유지)

### 이미지 비율을 유지하면서 resize하는 함수 정의

In [3]:
def maintain_proportion_and_resize_by_cv2(image_path, size=(416, 416)):

    # 이미지 읽기(BGR)
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"이미지를 열 수 없습니다: {image_path}")

    # 이미지 형식 변경하기(BGR -> RGB)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 높이, 너비 추출
    original_height, original_width = image.shape[:2]

    # 비율 계산
    ratio = min(size[0] / original_width, size[1] / original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)

    # 이미지 리사이즈
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # 새로운 이미지를 패딩하여 416x416 크기로 만들기
    new_image = np.full((size[1], size[0], 3), (255, 255, 255), dtype=np.uint8)  # 흰색 배경으로 초기화
    x_offset = (size[0] - new_width) // 2
    y_offset = (size[1] - new_height) // 2
    new_image[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return new_image


def maintain_proportion_and_resize_by_pil(image, size=(416, 416)):

    # PIL.Image 형식의 이미지를 비율을 유지하면서 지정된 크기로 리사이즈하고,
    # 흰색 배경으로 패딩하여 최종 이미지를 반환

    # 원본 이미지 크기 추출
    original_width, original_height = image.size

    # 비율 계산
    ratio = min(size[0] / original_width, size[1] / original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)

    # 이미지 리사이즈
    resized_image = image.resize((new_width, new_height), Img.ANTIALIAS)

    # 새로운 이미지를 패딩하여 지정된 크기로 만들기
    new_image = Img.new("RGB", size, (255, 255, 255))  # 흰색 배경으로 초기화
    x_offset = (size[0] - new_width) // 2
    y_offset = (size[1] - new_height) // 2
    new_image.paste(resized_image, (x_offset, y_offset))

    return new_image

### yolo(original_image) = yolo_output = bounding_box
- bounding_box = 데이터셋 구축을 위한 이미지
- 이미지 이름 = label(정답)
- original_image = 추후 결과를 보여주기 위해 계속 가지고 가야함.

In [4]:
# 라벨링이 되지 않은 이미지 경로 설정 -> yolo input 폴더의 경로

custom_image_path  = '/content/drive/MyDrive/yolo_efficientnet/datasets_for_unlabeled/'

In [5]:
# 이슈 해결 방식
# 클래스(키)의 인코딩 방식과 파일의 소분류 인코딩을 방식을 통일시킨다.
# 통일 시키는 방법 -> 한글 문자열을 자음 모음으로 분리하는 방법

import unicodedata
def normalize(text) :
    normalized_text = unicodedata.normalize('NFD', text)
    return normalized_text

In [9]:
# 클래스 라벨 사전 정의하기
classes_composition_list = ['니트','후드','맨투맨','셔츠블라우스','긴소매티셔츠','반소매티셔츠','민소매티셔츠','카라티셔츠','베스트','데님팬츠','슬랙스','트레이닝조거팬츠','숏팬츠','코튼팬츠','레깅스','와이드팬츠','후드집업','바람막이',
'코트','롱패딩','숏패딩','패딩베스트','블루종','레더자켓','무스탕','트러커자켓','블레이저','가디건','뽀글이후리스','사파리자켓','미니원피스','미디원피스','투피스','롱원피스', '점프수트','미니스커트','미디스커트','롱스커트']

cls_labels = {normalize(cls) : i for i, cls in enumerate(classes_composition_list)}

In [10]:
cls_labels

{'니트': 0,
 '후드': 1,
 '맨투맨': 2,
 '셔츠블라우스': 3,
 '긴소매티셔츠': 4,
 '반소매티셔츠': 5,
 '민소매티셔츠': 6,
 '카라티셔츠': 7,
 '베스트': 8,
 '데님팬츠': 9,
 '슬랙스': 10,
 '트레이닝조거팬츠': 11,
 '숏팬츠': 12,
 '코튼팬츠': 13,
 '레깅스': 14,
 '와이드팬츠': 15,
 '후드집업': 16,
 '바람막이': 17,
 '코트': 18,
 '롱패딩': 19,
 '숏패딩': 20,
 '패딩베스트': 21,
 '블루종': 22,
 '레더자켓': 23,
 '무스탕': 24,
 '트러커자켓': 25,
 '블레이저': 26,
 '가디건': 27,
 '뽀글이후리스': 28,
 '사파리자켓': 29,
 '미니원피스': 30,
 '미디원피스': 31,
 '투피스': 32,
 '롱원피스': 33,
 '점프수트': 34,
 '미니스커트': 35,
 '미디스커트': 36,
 '롱스커트': 37}

In [8]:
# 데이터셋 구축을 위해 yolo에서 탐지된 result를 모을 리스트 정의 -> result에는 바운딩 박스 포함
dataset_image_composition_list = []

# 데이터셋 구축을 위해 original_image 파일명에 포함된 label을 모을 리스트 정의
label_composition_list = []

# 탐지 성공 개수 정의
detect_success = 0

# 탐지 실패 개수 정의
detect_fail = 0

print(f'{len(os.listdir(custom_image_path))}개의 이미지를 분석하겠습니다.')



# custom_image_path에 있는 이미지를 하나씩 보면서
for image in tqdm(os.listdir(custom_image_path)):

    print(f'해당 이미지 파일 분석 중 : {image}')

    # original_image 정의 -> 객체 탐지에 실패했을 때는 bounding_box 대신 original_image를 input으로 만든다.
    natural_img = cv2.imread(custom_image_path + image)
    natural_img = cv2.cvtColor(natural_img, cv2.COLOR_BGR2RGB)

    # resize_image(yolo모델 인풋) 정의
    resized_img = maintain_proportion_and_resize_by_cv2(custom_image_path + image, (416, 416))
    resized_img_for_cropping = maintain_proportion_and_resize_by_cv2(custom_image_path + image, (416, 416))

    # label(소분류) 정의 -> 가정 : image = custom_image_path/대분류_소분류_image_숫자.jpg
    image_label = image.split('_')[1]
    image_label = normalize(image_label)
    # 인코딩/디코딩을 이용한 처리
    # image_label = image_label.encode('utf-8').decode('utf-8')


    # resized_image를 model의 인풋으로 넣고 output 정의
    result = yolo_model(resized_img)

    # 탐지 결과가 있으면
    if len(result.xyxy[0]) != 0:

        print(f'{image}의 탐지 결과를 출력합니다')

        # 탐지 결과 보여주기
        # result.show()

        # 탐지 성공 횟수 모으기
        detect_success += 1

        print(f'{image}의 bounding_box 추출 결과를 출력합니다')

        # bounding_box 추출 결과 확인
        for i, bounding_box in enumerate(result.xyxy[0]):
            xmin, ymin, xmax, ymax = map(int, bounding_box[:4])
            cls = int(bounding_box[5])
            confidence = round(bounding_box[4].item(), 2)

            # 이미지 자르기
            cropped_img = resized_img_for_cropping[ymin:ymax, xmin:xmax]

            # 클래스 이름
            class_name = result.names[cls]

            # crop된 이미지를 데이터셋으로 모으기
            dataset_image_composition_list.append(Img.fromarray(cropped_img, 'RGB'))


            # 라벨 모으기
            label_composition_list.append(cls_labels[image_label])


            print(f"탐지된 객체 {i+1}: 클래스 = {class_name}, 신뢰도 = {confidence * 100}%")

            # 자른 이미지를 그대로 사용 (추가 변환 필요 없음)
            # display(Img.fromarray(cropped_img, 'RGB'))

        print('------------------------------------------')

    # 탐지 결과가 없으면
    else:
        # 원본 이미지를 데이터셋으로 모으기
        dataset_image_composition_list.append(Img.fromarray(natural_img, 'RGB'))

        # 라벨 모으기
        label_composition_list.append(cls_labels[image_label])

        print(f'{image}의 탐지 결과가 없습니다.')
        print('------------------------------------------')

        # 탐지 실패 횟수 반영하기
        detect_fail += 1

print(f'탐지 성공 비율: {round(detect_success/len(glob.glob(custom_image_path)) * 100, 2)}')
print(f'탐지 실패 비율 : {round(detect_fail/len(glob.glob(custom_image_path)) * 100, 2)}')

# debugging 용
print(len(dataset_image_composition_list), len(label_composition_list))

import pickle
import time


# 음 뭔가 여기서 list에 넣을 때 사이즈 조정을 하고 데이터셋으로 넘겨야 하는 느낌
dataset_image_composition_list = [maintain_proportion_and_resize_by_pil(detected_image, (224, 22)) for detected_image in dataset_image_composition_list ]




Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 데이터를 파일로 저장
dataset = {
    'images': dataset_image_composition_list,
    'labels': label_composition_list
}

dataset_save_path = '/content/drive/MyDrive/yolo_efficientnet/effnet_dataset/train_dataset2.pkl'

with open(dataset_save_path, 'wb') as f:
    pickle.dump(dataset, f)

print("Dataset saved successfully.")

### 데이터셋 정의

- 필수 구현
    - __init__ : 생성자 정의
    - __len__ : 길이 정의
    - __getitem__ : 반환 형태 정의

In [ ]:
class EfficientNetDataset(Dataset):
    def __init__(self, dataset_image_composition_list, label_composition_list, transform=None):

        self.dataset_image_composition_list = dataset_image_composition_list

        self.label_composition_list = label_composition_list

        self.transform = transform

    def __len__(self):

        return len(self.dataset_image_composition_list)

    def __getitem__(self, idx):

        image = self.dataset_image_composition_list[idx]

        label = self.label_composition_list[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

### transform 정의
- transform 해야 하는 이유
    - 데이터 표준화: 모델에 입력되는 이미지의 크기, 형태 등을 일정하게 맞춰주는 작업
    - 데이터 증강: 학습 데이터셋의 다양성을 높여 모델의 일반화 성능 높이기
    - 텐서 변환: 이미지를 PyTorch에서 사용하는 텐서 형태로 변환

In [ ]:
transform = transforms.Compose([
    # 직접 정의한 resize, crop 함수 적용
    transforms.Lambda(lambda image : maintain_proportion_and_resize_by_pil(image, size = (224, 224))),
    # 랜덤하게 이미지를 좌우 반전 -> 일반화 성능 향상
    transforms.RandomHorizontalFlip(),
    # 이미지를 랜덤하게 10도 이내로 회전 -> 일반화 성능 향상
    transforms.RandomRotation(10),
    # 밝기, 대비, 채도, 색조 조정 -> 다양한 조명 아래에서 찍히는 옷 이미지 특성 상 필수적..
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    # 이미지를 텐서로 변환
    transforms.ToTensor(),
    # 이미지 정규화 -> 일반화 성능 향상
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### effnet input들 저장하기

In [ ]:
import pickle
from collections import Counter

dataset_save_path = '/content/drive/MyDrive/yolo_efficientnet/effnet_dataset/train_dataset2.pkl'

# 저장한 데이터셋 불러오기
with open(dataset_save_path, 'rb') as f:
    dataset = pickle.load(f)

dataset_image_composition_list = dataset['images']
label_composition_list = dataset['labels']


In [ ]:
import os
from tqdm import tqdm

# 저장할 디렉토리 설정
output_directory = '/content/drive/MyDrive/yolo_efficientnet/effnet_dataset/actual_dataset/'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 각 PIL 이미지를 고유한 파일 이름으로 저장
for i, img in enumerate(tqdm(dataset_image_composition_list)):
    # 파일 이름에 사용할 레이블을 가져오기 위해 label_composition_list를 참조
    output_path = os.path.join(output_directory, f'{label_composition_list[i]}_image_{i}.jpg')
    img.save(output_path)
    print(f'Image saved to {output_path}')




### 데이터셋 생성 / 데이터로더 생성 및 정의

In [ ]:
from sklearn.model_selection import train_test_split
import pickle
from collections import Counter
import random

def make_dataloader(dataset_image_composition_list,label_composition_list, epoch, batch_size) :

    # dataset_image_composition_list, label_composition_list 을 train과 test로 분리하기
    train_image_composition_list, valid_image_composition_list, train_label_composition_list, valid_label_composition_list = train_test_split(
        dataset_image_composition_list,
        label_composition_list,
        test_size = 0.2,
        random_state = epoch)

    # 개수 스케일링 숫자 도출하기 : train
    train_mini_num = min(Counter(train_label_composition_list).values())
    train_scaling_num = int(train_mini_num * 0.95)

    # 개수 스케일링 숫자 도출하기 : valid
    valid_mini_num = min(Counter(valid_label_composition_list).values())
    valid_scaling_num = int(valid_mini_num * 0.95)


    train_images = []
    train_labels = []

    for i in range(38) :
        target_image_list = [train_image_composition_list[j] for j, label in enumerate(train_label_composition_list) if label == i]
        target_label_list = [label for j, label in enumerate(train_label_composition_list) if label == i]
        sample_image_list = random.sample(target_image_list, k = train_scaling_num)
        sample_label_list = random.sample(target_label_list, k = train_scaling_num)
        train_images.extend(sample_image_list)
        train_labels.extend(sample_label_list)

    # valid 라벨 별 개수 맞추기
    valid_images = []
    valid_labels = []

    for i in range(38) :
        target_image_list = [valid_image_composition_list[j] for j, label in enumerate(valid_label_composition_list) if label == i]
        target_label_list = [label for j, label in enumerate(valid_label_composition_list) if label == i]
        sample_image_list = random.sample(target_image_list, k = valid_scaling_num)
        sample_label_list = random.sample(target_label_list, k = valid_scaling_num)
        valid_images.extend(sample_image_list)
        valid_labels.extend(sample_label_list)

    # train dataset 정의하기
    train_dataset = EfficientNetDataset(train_images, train_labels, transform=transform)

    # valid dataset 정의하기
    valid_dataset = EfficientNetDataset(valid_images, valid_labels, transform=transform)

    # 배치 사이즈 설정
    batch_size  = batch_size

    # train_dataloader 정의하기
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # valid dataloader 정의하기
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)


    return train_loader, valid_loader

### EfficientNet 로드 및 학습

In [ ]:
# device 정의
device = torch.device('cuda'if torch.cuda.is_available else 'cpu')

# pytorch에 있는 사전 학습 EfficientNet 모델 불러오기
effnet_v2_s_model = get_model.efficientnet_v2_s(pretrained=True)

# effnet_v2_s_model의 연산 공간 device 정의
effnet_v2_s_model.to(device)

In [ ]:
# 클래스 수에 맞게 out_features 조정하기
num_class = 38

print('[변경 전]')
print(effnet_v2_s_model.classifier)

effnet_v2_s_model.classifier[1] = torch.nn.Linear(effnet_v2_s_model.classifier[1].in_features, num_class)

print('[변경 후]')
print(effnet_v2_s_model.classifier)





[변경 전]
Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)
[변경 후]
Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=38, bias=True)
)


In [ ]:
# 이전까지 학습했던 파라미터 파일 적용하기

eff_pt_root_path = '/content/drive/MyDrive/yolo_efficientnet/efficientnet_train.pt/'
eff_pt_name = f'trained_{90}_effnet.pt'

checkpoint = torch.load(eff_pt_root_path + eff_pt_name)

# Load the model state dictionary
effnet_v2_s_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
print(effnet_v2_s_model.classifier)

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=38, bias=True)
)


In [ ]:
# 손실함수, 옵티마이저, 스케쥴러 정의
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(effnet_v2_s_model.parameters(), lr=0.001)
# scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=10,
#                        pct_start=0.3, anneal_strategy='cos', div_factor=25.0, final_div_factor=10000.0)

# max_lr : 최대 학습률 -> 초기 학습률은 크게하고 점차 감소시키기 위해서 높은 값을 사용한다.
# steps_per_epoch=len(train_loader): 한 에포크당 스텝 수를 설정
# epochs=10: 전체 에폭 수와 동일하게 설정한다. 몇 번의 학습동안 스케쥴링할 지 결정
# pct_start=0.3: 전체 학습 기간 중 학습률이 증가하는 비율을 설정(30% : 증가, 70% : 감소)
# anneal_strategy='cos': 학습률을 코사인 함수 형태로 감소
# div_factor=25.0: 초기 학습률을 max_lr에 나누는 값(초기 학습률 = max_lr / 25)
# final_div_factor=10000.0: 최종 학습률을 초기 학습률에 나누는 값(최종 학습률 = 초기 학습률 / 10000)

In [ ]:
# effnet_v2_s_model.to(device) # 혹시라도 device가 안맞다는 에러 발생 시 실행할 것
print('한번 더 확인하기')
print(effnet_v2_s_model.classifier)

한번 더 확인하기
Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=38, bias=True)
)


In [ ]:
from tqdm import tqdm

# 학습 횟수
epochs = 400
effnet_v2_s_model.to(device)
for epoch in range(91, epochs):
    # 한 에폭에서 학습모드 검증모드를 왔다갔다 하기 때문에 반복문 안에서 학습모드 선언
    effnet_v2_s_model.train()

    train_loss = 0.0
    train_total = 0
    train_correct = 0

    batch_size = 32

    train_loader, valid_loader = make_dataloader(dataset_image_composition_list,label_composition_list, epoch, batch_size)

    scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch =len(train_loader) , epochs=10,
                    pct_start=0.3, anneal_strategy='cos', div_factor=25.0, final_div_factor=10000.0)

    for inputs, labels in tqdm(train_loader):
        # 계산할 디바이스 정의하기
        inputs, labels = inputs.to(device), labels.to(device)
        # optimizer가 이전 결과에 대한 미분값을 가지고 있으면 가중치를 이상하게 학습시킬 가능성이 있으므로
        optimizer.zero_grad()

        # 모델에 inputs을 넣어서 outputs을 산출받고
        outputs = effnet_v2_s_model(inputs)
        # 모델 결과와 실제 정답을 비교해서 loss를 계산하고
        loss = loss_func(outputs, labels)
        # 역전파를 진행하고
        loss.backward()
        # 가중치를 업데이트하고
        optimizer.step()
        # 각 배치에서 학습률을 조정한다.
        scheduler.step()

        # tensor에 있는 값을 item()을 통해 숫자로 꺼내와서 각 이미지와 라벨에 대한 손실을 모두 누적하고
        train_loss += loss.item()


        _, predicted = torch.max(outputs, 1)
        predict_list = predicted.tolist()


        # 각 배치의 전체 개수
        train_total += labels.size(0)
        # 각 배치의 맞춘 것의 개수
        train_correct += (predicted == labels).sum().item()

    # 손실을 평균값으로 계산
    epoch_loss = train_loss / len(train_loader)
    # 전체 이미지에 대해서 분류를 성공한 것의 비율을 구한다.
    epoch_accuracy = (train_correct / train_total) * 100

    print(f'Train_Epoch [{epoch+1}/{epochs}], Train_Loss: {epoch_loss:.4f}, Train_Accuracy: {epoch_accuracy:.2f}%')

    # record_by_scheduler.append((epoch_loss, epoch_accuracy))

    # 검증 모드 설정하기
    effnet_v2_s_model.eval()

    valid_loss = 0.0
    valid_total = 0
    valid_correct = 0

    # 검증 단계에서는 기울기 계산을 하지 않으므로
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = effnet_v2_s_model(inputs)

            loss = loss_func(outputs, labels)
            # 검증 모드이기 때문에
            # 역전파 X
            # 파라미터 업데이트 X
            # 스케쥴러 업데이트 X

            valid_loss += loss.item()

            _, predicted = torch.max(outputs, 1)


            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()

    # 검증 손실을 평균값으로 계산
    valid_loss = valid_loss / len(valid_loader)
    # 검증 데이터에서의 정확도 계산
    valid_accuracy = (valid_correct / valid_total) * 100

    print(f'Valid_Epoch [{epoch+1}/{epochs}], Valid_Loss: {valid_loss:.4f}, Valid_Accuracy: {valid_accuracy:.2f}%')



    # 5의 배수 에폭당 파라미터 저장하기
    if (epoch + 1) % 10 == 0 :
        final_effnet_pt_save_path = f'/content/drive/MyDrive/yolo_efficientnet/efficientnet_train.pt/trained_{epoch + 1}_effnet.pt'
        torch.save( {'model_state_dict': effnet_v2_s_model.state_dict()}, final_effnet_pt_save_path)



  0%|          | 0/123 [00:00<?, ?it/s]<ipython-input-2-49375529a06c>:45: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize((new_width, new_height), Img.ANTIALIAS)
 99%|█████████▉| 122/123 [01:12<00:00,  1.62it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 123/123 [01:12<00:00,  1.69it/s]


Train_Epoch [92/400], Train_Loss: 0.7165, Train_Accuracy: 77.36%
Valid_Epoch [92/400], Valid_Loss: 0.8487, Valid_Accuracy: 72.93%


100%|██████████| 130/130 [01:17<00:00,  1.68it/s]


Train_Epoch [93/400], Train_Loss: 0.6747, Train_Accuracy: 78.05%
Valid_Epoch [93/400], Valid_Loss: 0.7994, Valid_Accuracy: 76.32%


100%|██████████| 129/129 [01:16<00:00,  1.69it/s]


...

Train_Epoch [259/400], Train_Loss: 0.2172, Train_Accuracy: 92.52%
Valid_Epoch [259/400], Valid_Loss: 0.2057, Valid_Accuracy: 93.23%


100%|██████████| 124/124 [01:13<00:00,  1.69it/s]


Train_Epoch [260/400], Train_Loss: 0.2198, Train_Accuracy: 92.36%
Valid_Epoch [260/400], Valid_Loss: 0.1746, Valid_Accuracy: 94.15%


 26%|██▌       | 32/124 [00:19<00:55,  1.65it/s]


KeyboardInterrupt: 

### 학습된 파라미터 저장하기

In [ ]:
# final_effnet_pt_save_path = '/content/drive/MyDrive/yolo_efficientnet/efficientnet_train.pt/trained_effnet.pt'

# torch.save( {'model_state_dict': effnet_v2_s_model.state_dict()},final_effnet_pt_save_path)